# Jackknife correlation: 
###En algunas ocasiones se presenta dificultad para encontrar estimadores de un parámetro $\theta$ que posean propiedades deseables como insesgamiento y precisión; además, cuando se dispone de un estimador sesgado, generalmente no se dispone de metodología para la estimación del sesgo. Otras veces, se dispone de un estimador insesgado, pero no se conoce una expresión cerrada para su varianza o para un estimador de la misma, lo cual obliga a realizar aproximaciones de distinta naturaleza, las cuales no producen los resultados más deseables. 
#Descripción del método: 
###Descripción del método: 
#Ejemplo: 
###Un equipo de investigadores quiere estudiar si existe correlación en la presencia de dos sustancias (A y B) en el agua de los ríos. Para ello han realizado una serie de mediciones en las que se cuantifica la concentración de las dos sustancias en 10 muestras independientes de agua. Se sospecha que el instrumento de lectura sufre alguna avería que provoca que algunas lecturas se disparen, por esta razón se quiere emplear un método de correlación robusto. El objetivo de este ejemplo es ilustrar el método de Jackknife, por lo que se asume que se cumplen las condiciones para la correlación de Pearson. 
#Datos: 


In [ ]:
# Datos simulados de dos variables A y B
a = np.array([12,9,6,7,2,5,4,0,1,8])
b = np.array([3,5,1,9,5,3,7,2,10,5])

# Se introduce un outlier
a[5] = 20
b[5] = 16
# Gráfico
# ==============================================================================
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(a, label='A')
ax.plot(b, label='B')
ax.set_xlabel('ID muestra')
ax.set_ylabel('Concentración')
ax.set_title('Concentración sustancias A y B en las muestras')
x.legend();


# Correlación de Pearson: 
###Se procede a calcular la correlación de Pearson con y sin el outlier.


In [ ]:
# Correlación con outlier
r, p = stats.pearsonr(a, b)
print(f'Correlación Pearson con outlier: r={r}, p-value={p}')
# Correlación sin outlier
r, p = stats.pearsonr(np.delete(a, 5), np.delete(b, 5))
print(f'Correlación Pearson sin outlier: r={r}, p-value={p}')


### Correlación Pearson con outlier: r=0.517373115168915, p-value=0.12563522982639538
### Correlación Pearson sin outlier: r=-0.18420184544326057, p-value=0.6351961086690547
### Se confirma que, La observación número 5, tiene una gran influencia en el resultado de la correlación, siendo de 0.52 si está presente y de -0.18 si se excluye.
# Jackknife Pearson correlation


In [ ]:
# Función Jackknife correlation
# ==============================================================================
def correlacion_jackknife(x, y):
    '''
    Esta función aplica el método de Jackknife para el cálculo del coeficiente
    de correlación de Pearson.
    Parameters
    ----------
    x : 1D np.ndarray, pd.Series 
        Variable X.
    y : 1D np.ndarray, pd.Series
        Variable y.     
    Returns 
    -------
    correlaciones: 1D np.ndarray
        Valor de correlación para cada iteración de Jackknife
    '''
    n = len(x)
    valores_jackknife = np.full(shape=n, fill_value=np.nan, dtype=float)
    for i in range(n):
        # Loop para excluir cada observación y calcular la correlación
        r = stats.pearsonr(np.delete(x, i), np.delete(y, i))[0]
        valores_jackknife[i] = r
    promedio_jackknife = np.nanmean(valores_jackknife)
    standar_error = np.sqrt(((n - 1) / n) * \n                    np.nansum((valores_jackknife - promedio_jackknife) ** 2))
    bias = (n - 1) * (promedio_jackknife - stats.pearsonr(x, y)[0])
    resultados = {
        'valores_jackknife' : valores_jackknife,
        'promedio'          : promedio_jackknife,
        'se'                : standar_error,
        'bias'              : bias
    }
    return resultados
correlacion = correlacion_jackknife(x=a, y=b)
print(f'Correlación jackknife: {correlacion['promedio']}')
print(f'Error estándar: {correlacion['se']}')
print(f'Error bias: {correlacion['bias']}')
print(f'Valores_jackknife: {correlacion['valores_jackknife']}'


### Correlación jackknife: 0.4781856690553491
### Error estándar: 0.6915097651492329
### Error bias: -0.35268701502209354
### Valores_jackknife: [ 0.64719522  0.53705998  0.54597653  0.52827609  0.51215595 -0.18420185  0.53554935  0.44125573  0.69065085  0.52793883]
### Para identificar si existe algún valor muy influyente, se puede graficar el cambio que produce en el coeficiente de regresión la exclusión de cada observación.


In [ ]:
variacion_corr = correlacion['valores_jackknife'] - stats.pearsonr(a, b)[0]
fig, ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(variacion_corr)
ax.set_xlabel('ID muestra')
ax.set_ylabel('Variación correlación');


### El método Jackknife correlation solo ha sido capaz de amortiguar una pequeña parte de la influencia del outlier, sin embargo, sí ha permitido identificar qué observación está afectando en mayor medida.
# Ejemplo matriz de correlaciones: 
### Cuando se dispone de múltiples variables numéricas, por ejemplo en problemas de modelado estadístico y machine learning, es conveniente estudiar el grado de correlación entre las variables disponibles.
### Una forma de hacerlo es mediante matrices de correlación, en las que se muestra el coeficiente de correlación para cada par de variables.


In [ ]:
# Datos
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/' +
       'Estadistica-machine-learning-python/master/data/SaratogaHouses.csv')
datos = pd.read_csv(url, sep=)
# Se renombran las columnas para que sean más descriptivas
datos.columns = ['precio', 'metros_totales', 'antiguedad', 'precio_terreno',
                 'metros_habitables', 'universitarios', 'dormitorios', 
                 'chimenea', 'banyos', 'habitaciones', 'calefaccion',
                 'consumo_calefacion', 'desague', 'vistas_lago',
                 'nueva_construccion', 'aire_acondicionado']
# Variables numéricas
datos = datos.select_dtypes(include=['float64', 'int'])
# Matriz de correlación
# ==============================================================================    
corr_matrix = datos.corr(method='pearson')
corr_matrix


### Las matrices de correlación tienen el inconveniente de tener un tamaño notable cuando se dispone de muchas variables.
### Para facilitar la identificación de pares de variables con correlaciones altas, es conveniente convertirlas en formato de tabla larga (tidy).


In [ ]:
def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matriz de correlación de pandas en formato tidy.
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    return(corr_mat)
tidy_corr_matrix(corr_matrix).head(10)
# Heatmap matriz de correlaciones
# ==============================================================================
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5, 5))
sns.heatmap(
    corr_matrix,
    annot     = True,
    cbar      = False,
    annot_kws = {'size': 8},
    vmin      = -1,
    vmax      = 1,
    center    = 0,
    cmap      = sns.diverging_palette(20, 220, n=200),
    square    = True,
    ax        = ax
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right',
)
ax.tick_params(labelsize = 10)
